# Segmenting and Clustering Neighborhoods in Toronto

## Packages

### Install Packages

In [1]:
#pip install beautifulsoup4
#pip install lxml
#pip install requests

### Import Packages

In [4]:
from bs4 import BeautifulSoup as bsoup
import requests as req
import csv
import pandas as pd
import numpy as np

## Data

In [5]:
#Get information from Wikipedia

source = req.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bsoup(source, 'lxml')
#print(soup.prettify())

# make the first table in the page into a variable
table = soup.table
#table

## Create the DataFrame

### Create Headers

In [6]:
# Get headers from table
row_th = table.find_all('th')

# Make them into a string, and clean out the html code, the brackets, etc.
str_headers = str(row_th)
cleanheader = bsoup(str_headers, "lxml").get_text()
cleanheader = cleanheader.replace("[","")
cleanheader = cleanheader.replace("\n]","")

# Make the resulting string into a list
cleanheader = list(cleanheader.split(", "))

### Create Rows

In [7]:
# Get all the rows in that table, separately
rows = table.find_all('tr')
#print(rows)

# Make an empty variable
list_rows = []

# Fill the variable with each table row, removing brackets, etc., and making it into a list
for row in rows:
    row_td = row.find_all('td')
    str_cells = str(row_td)
    cleantext = bsoup(str_cells, "lxml").get_text()
    cleantext = cleantext.replace("[","")
    cleantext = cleantext.replace("\n]","")
    cleantext = list(cleantext.split(", "))
    list_rows.append(cleantext)
#print(cleantext)

# Put each row into a row in a dataframe
df = pd.DataFrame(list_rows)
df = df.drop(0)  ## drop the top row, which belongs to the 'th' headers

### Assign Column Names and Drop Missing Boroughs

In [8]:
df.columns = cleanheader
df.head()

# drop "Not assigned" boroughs
df = df[df.Borough != 'Not assigned']

### Rename Missing Neighbourhoods with Borough Name

In [9]:
values = df['Borough']
df['Neighbourhood'] = values.where(df.Neighbourhood == "Not assigned", other=df.Neighbourhood)

### Combine Multiple Neighborhoods in Postcode

In [10]:
df = df.groupby(['Postcode','Borough'],sort=False).agg(', '.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df.to_csv(r'torontosegs.csv')

In [12]:
df.shape

(103, 3)

In [13]:
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
df = df.merge(coord, left_on='Postcode', right_on='Postal Code')
df.drop(['Postal Code'], axis=1)
df.head()

,Postcode,Borough,Neighbourhood,Postal Code_x,Latitude_x,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,M3A,43.753259,-79.329656,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,M4A,43.725882,-79.315572,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,M5A,43.654260,-79.360636,M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,M6A,43.718518,-79.464763,M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,M7A,43.662301,-79.389494,M7A,43.662301,-79.389494


# Cluster Neighborhoods in Toronto

## Install Packages

In [15]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0

## Import Packages

In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Get Toronto coordinates

In [56]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


## Create map of Toronto with neighbourhoods marked

In [23]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Foursquare data

In [24]:
CLIENT_ID = 'IA44NH1GVQV5OTMYIXR4Q2LHHWHTCGHURWHPECLJ3IB3CAMB' # your Foursquare ID
CLIENT_SECRET = 'JDHQIX3KV4MVGKBU4KK31ADXKH2FWR5XY5G4MJGVE3VYH3VV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IA44NH1GVQV5OTMYIXR4Q2LHHWHTCGHURWHPECLJ3IB3CAMB
CLIENT_SECRET:JDHQIX3KV4MVGKBU4KK31ADXKH2FWR5XY5G4MJGVE3VYH3VV


### First Neighbourhood

In [29]:
df.loc[0, 'Neighbourhood']

'Parkwoods'

In [28]:
neighbourhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))
radius = 500
print("radius =", radius)

LIMIT = 100
print("LIMIT =",LIMIT)

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.
radius = 500
LIMIT = 100


In [33]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
results = req.get(url).json()
#results
    # all of the information is in the items key

#### Venues in Parkwood

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Cafe Plenty,Café,43.654571,-79.389450
4,Tsujiri,Tea Room,43.655374,-79.385354


### All Toronto Neighbourhoods

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = req.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

#### Venues in all Toronto Neighbourhoods

In [48]:
print(toronto_venues.shape)
toronto_venues[['Neighbourhood','Venue Category','Venue']].head(15)

(2253, 7)


,Neighbourhood,Venue Category,Venue
0,Parkwoods,Park,Brookbanks Park
1,Parkwoods,Fast Food Restaurant,KFC
2,Parkwoods,Food & Drink Shop,Variety Store
3,Victoria Village,Hockey Arena,Victoria Village Arena
4,Victoria Village,Coffee Shop,Tim Hortons
5,Victoria Village,Portuguese Restaurant,Portugril
6,Victoria Village,Intersection,Eglinton Ave E & Sloane Ave/Bermondsey Rd
7,"Harbourfront, Regent Park",Bakery,Roselle Desserts
8,"Harbourfront, Regent Park",Coffee Shop,Tandem Coffee
9,"Harbourfront, Regent Park",Gym / Fitness Center,Toronto Cooper Koo Family Cherry St YMCA Centre


In [52]:
#toronto_venues.groupby('Neighbourhood').count()
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 280 unique categories.


## Analyze each Neighbourhood

### Proportion of Each Venue over Neighbourhoods

In [53]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,W

### Find Most Popular Venues in each Neighbourhood

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(50)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,American Restaurant,Burger Joint,Bakery,Cosmetics Shop,Hotel
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Asian Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Pizza Place,Sandwich Place,Beer Store,Coffee Shop,Liquor Store,Fried Chicken Joint,Video Store,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pub,Sandwich Place,Pool,Skating Rink,Dance Studio,Pharmacy,Gym,Dumpling Restaurant
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Deli / Bodega,Fast Food Restaurant,Bank,Supermarket,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Diner,Middle Eastern Restaurant
6,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Grocery Store,Butcher,Sushi Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Sandwich Place
8,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Cheese Shop,Café,Farmers Market,Bakery,Seafood Restaurant,Steakhouse,Beer Bar
9,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


# Cluster Neighbourhoods

### Algorithm

In [131]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:7] 

array([0, 0, 1, 0, 0, 0, 3])

### Dataframe with cluster label and top 10 venues for each neighbourhood

In [134]:
#neighbourhoods_venues_sorted = neighbourhoods_venues_sorted.drop(columns=['Cluster Labels'])

In [135]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# perform an inner join toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how = 'inner')

toronto_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Postal Code_x,Latitude_x,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656,M3A,43.753259,-79.329656,M3A,43.753259,-79.329656,1,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572,M4A,43.725882,-79.315572,M4A,43.725882,-79.315572,0,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dessert Shop
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,M5A,43.654260,-79.360636,M5A,43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Café,Farmers Market
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,M6A,43.718518,-79.464763,M6A,43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,M7A,43.662301,-79.389494,M7A,43.662301,-79.389494,0,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
6,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,M1B,43.806686,-79.194353,M1B,43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,M3B,North York,Don Mills North,M3B,43.745906,-79.352188,M3B,43.745906,-79.352188,M3B,43.745906,-79.352188,0,Café,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,M4B,East York,"Woodbine Gardens, Parkview Hill",M4B,43.706397,-79.309937,M4B,43.706397,-79.309937,M4B,43.706397,-79.309937,0,Fast Food Restaurant,Pizza Place,Pet Store,Gastropub,Café,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson, Garden District",M5B,43.657162,-79.378937,M5B,43.657162,-79.378937,M5B,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Ramen Restaurant,Bakery,Fast Food Restaurant,Pizza Place
10,M6B,North York,Glencairn,M6B,43.709577,-79.445073,M6B,43.709577,-79.445073,M6B,43.709577,-79.445073,1,Park,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


### Visualizing Clusters

In [136]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 0: Coffee Shops and Cafes

In [137]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,-79.315572,M4A,43.725882,-79.315572,M4A,43.725882,-79.315572,0,Intersection,Hockey Arena,Coffee Shop,Portuguese Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Empanada Restaurant,Electronics Store,Dessert Shop
2,Downtown Toronto,-79.360636,M5A,43.654260,-79.360636,M5A,43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Breakfast Spot,Mexican Restaurant,Café,Farmers Market
3,North York,-79.464763,M6A,43.718518,-79.464763,M6A,43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Women's Store,Boutique,Fraternity House,Event Space,Miscellaneous Shop,Coffee Shop,Accessories Store,Vietnamese Restaurant
4,Queen's Park,-79.389494,M7A,43.662301,-79.389494,M7A,43.662301,-79.389494,0,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant,Persian Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
7,North York,-79.352188,M3B,43.745906,-79.352188,M3B,43.745906,-79.352188,0,Café,Gym / Fitness Center,Caribbean Restaurant,Baseball Field,Japanese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
8,East York,-79.309937,M4B,43.706397,-79.309937,M4B,43.706397,-79.309937,0,Fast Food Restaurant,Pizza Place,Pet Store,Gastropub,Café,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center
9,Downtown Toronto,-79.378937,M5B,43.657162,-79.378937,M5B,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Ramen Restaurant,Bakery,Fast Food Restaurant,Pizza Place
12,Scarborough,-79.160497,M1C,43.784535,-79.160497,M1C,43.784535,-79.160497,0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
13,North York,-79.340923,M3C,43.725900,-79.340923,M3C,43.725900,-79.340923,0,Gym,Asian Restaurant,Beer Store,Coffee Shop,Dim Sum Restaurant,Bike Shop,Japanese Restaurant,Italian Restaurant,Restaurant,Chinese Restaurant
14,East York,-79.318389,M4C,43.695344,-79.318389,M4C,43.695344,-79.318389,0,Pharmacy,Athletics & Sports,Cosmetics Shop,Curling Ice,Bus Stop,Skating Rink,Beer Store,Park,Gluten-free Restaurant,Gift Shop


### Cluster 1: Parks

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,-79.329656,M3A,43.753259,-79.329656,M3A,43.753259,-79.329656,1,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,Scarborough,-79.194353,M1B,43.806686,-79.194353,M1B,43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
10,North York,-79.445073,M6B,43.709577,-79.445073,M6B,43.709577,-79.445073,1,Park,Pizza Place,Japanese Restaurant,Pub,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
21,York,-79.453512,M6E,43.689026,-79.453512,M6E,43.689026,-79.453512,1,Park,Pharmacy,Market,Fast Food Restaurant,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Dim Sum Restaurant
35,East York,-79.338106,M4J,43.685347,-79.338106,M4J,43.685347,-79.338106,1,Park,Coffee Shop,Convenience Store,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,North York,-79.464763,M3K,43.737473,-79.464763,M3K,43.737473,-79.464763,1,Park,Airport,Other Repair Shop,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
49,North York,-79.490074,M6L,43.713756,-79.490074,M6L,43.713756,-79.490074,1,Park,Bakery,Construction & Landscaping,Deli / Bodega,Basketball Court,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
61,Central Toronto,-79.388790,M4N,43.728020,-79.388790,M4N,43.728020,-79.388790,1,Park,Bus Line,Swim School,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
66,North York,-79.400049,M2P,43.752758,-79.400049,M2P,43.752758,-79.400049,1,Park,Convenience Store,Bank,Bar,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
77,Etobicoke,-79.554724,M9R,43.688905,-79.554724,M9R,43.688905,-79.554724,1,Pizza Place,Park,Mobile Phone Shop,Bus Line,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### Cluster 2: Bank

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,-79.554724,M9B,43.650943,-79.554724,M9B,43.650943,-79.554724,2,Bank,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner


### Cluster 3: Bank and International Restaurants

In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,-79.554724,M9B,43.650943,-79.554724,M9B,43.650943,-79.554724,3,Bank,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner
39,North York,-79.385975,M2K,43.786947,-79.385975,M2K,43.786947,-79.385975,3,Japanese Restaurant,Café,Bank,Chinese Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio


### Cluster 4: Cafeterias

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,-79.374714,M2L,43.75749,-79.374714,M2L,43.75749,-79.374714,4,Cafeteria,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Diner


### Cluster 5: Parks and Asian Restaurants

In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,York,-79.518188,M9N,43.706876,-79.518188,M9N,43.706876,-79.518188,5,Park,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
68,Central Toronto,-79.411307,M5P,43.696948,-79.411307,M5P,43.696948,-79.411307,5,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
91,Downtown Toronto,-79.377529,M4W,43.679563,-79.377529,M4W,43.679563,-79.377529,5,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


### Cluster 6: Playgrounds

In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,-79.239476,M1J,43.744734,-79.239476,M1J,43.744734,-79.239476,6,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
38,Scarborough,-79.262029,M1K,43.727929,-79.262029,M1K,43.727929,-79.262029,6,Department Store,Playground,Discount Store,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant,Donut Shop
83,Central Toronto,-79.383160,M4T,43.689574,-79.383160,M4T,43.689574,-79.383160,6,Tennis Court,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


### Cluster 7: Empty

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
